In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import os, h5py
from pathlib import Path
import time
import datetime
from collections import OrderedDict

import holoviews as hv
from holoviews.operation.datashader import datashade
from holoviews.streams import PlotSize
import datashader as ds
import datashader.transfer_functions as tf

PlotSize.scale=2
hv.extension('plotly', 'bokeh', 'matplotlib')

 ## Use Dummy hdf to get error over time steps

In [ ]:
# filepath to hdf file
modelHDF_path = Path(r'C:\Users\christianl\repos\WF_Analysis\read_hdf\WF_WestForkCalcasieu.p05.hdf')

In [ ]:
# creates variable of hdf file that will be read
hdfFile = h5py.File(modelHDF_path,'r')

# sets path within hdf file to find simulation window
hdfplan_general = hdfFile['Plan Data']['Plan Information'].attrs

# converts plan infomration to pandas series
# planDatageneral = pd.Series(hdfplan_general).map(lambda st: st.decode('UTF-8'))
# this will not work for models run with adjusted time step
# this will convert paln info to pandas series and decode b strings while skiping integers and floats and shit 
planDatageneral = pd.Series(hdfplan_general).map(lambda x: x.decode('UTF-8') if isinstance(x, bytes) else x)


# sets path within hdf file to find geometry information
hdfpaln_geometryData = hdfFile['Geometry']['2D Flow Areas']['Attributes']

# converts 2D Flow Area Information to dataframe
planDataGeometry = np.array(hdfpaln_geometryData)
planDataGeometry = pd.DataFrame(planDataGeometry)

In [ ]:
planDatageneral

In [ ]:
int(planDataGeometry['Cell Count'])

In [ ]:
# "picks" the 2D Area name out of the geometry data
geometry_2DName = str(planDataGeometry['Name'][0]).split("'")[1]
geometry_2DName

In [ ]:
hdfplan_ComputeMessage = hdfFile['Results']['Summary']['Compute Messages (text)']
plan_ComputeMessage = pd.Series(hdfplan_ComputeMessage).map(lambda x: x.decode('ascii'))

In [ ]:
print(plan_ComputeMessage)

In [ ]:
# sets path within hdf file to find the computed timestamps and timesteps in the computation block
hdfplan_OutTimeDateStamp = hdfFile['Results']['Unsteady']['Output']['Output Blocks']['Computation Block']['Global']['Time Date Stamp (ms)']
hdfplan_OutTimeDateStep = hdfFile['Results']['Unsteady']['Output']['Output Blocks']['DSS Hydrograph Output']['Unsteady Time Series']['Time Step']

# converts plan infomration to pandas dataframe
planDataOutTimeDateStamp = pd.Series(hdfplan_OutTimeDateStamp).map(lambda x: x.decode('ascii'))
planDataOutTimeDateStep = pd.DataFrame(hdfplan_OutTimeDateStep, columns=['Seconds'])

# convert decoded series to dataframe
planDataOutTimeDateStamp_df = planDataOutTimeDateStamp.to_frame()

# converts HEC-RAS's SAS format to a normal fucking timestamp
planDataOutTimeDateStamp_df[0] = pd.to_datetime(planDataOutTimeDateStamp_df[0], format= "%d%b%Y %H:%M:%S:%f")

In [ ]:
planDataOutTimeDateStep

In [ ]:
# planDataOutTimeDateStamp_df['time Step (Computed)'] = planDataOutTimeDateStamp_df[0]
timeDiff = [] 
for idx, row in planDataOutTimeDateStamp_df.iterrows():
    try:
        timeDiffComp = planDataOutTimeDateStamp_df[0][idx] - planDataOutTimeDateStamp_df[0][idx-1]
        timeDiff.append(timeDiffComp)
    except:
        timeDiffComp = 0
        timeDiff.append(timeDiffComp)
planDataOutTimeDateStamp_df['Time Step'] = pd.Series(timeDiff)

In [ ]:
planDataOutTimeDateStamp_df

In [ ]:
# sets path within hdf file to find the 2D Iterations and 2D Itteration Error
hdfplan_Out2DItterError = hdfFile['Results']['Unsteady']['Output']['Output Blocks']['Computation Block']['2D Global']['2D Iteration Error']
hdfplan_Out2DItter = hdfFile['Results']['Unsteady']['Output']['Output Blocks']['Computation Block']['2D Global']['2D Iterations']

# converts plan infomration to pandas dataframe
planDataOut2DItterError = pd.DataFrame(hdfplan_Out2DItterError, columns=['Error'])
planDataOut2DItter = pd.DataFrame(hdfplan_Out2DItter, columns=['Iterations', 'Random Boolean', 'Cell'])

In [ ]:
planDataOut2DItterError

In [ ]:
runtimeAnalysis_df = pd.DataFrame()

In [ ]:
runtimeAnalysis_df['Time Stamp'] = planDataOutTimeDateStamp_df[0]
runtimeAnalysis_df['Time Step'] = planDataOutTimeDateStamp_df['Time Step']
runtimeAnalysis_df['Iterations'] = planDataOut2DItter['Iterations']
runtimeAnalysis_df['Error'] = planDataOut2DItterError['Error']
runtimeAnalysis_df['Cell'] = planDataOut2DItter['Cell']

In [ ]:
# set cells marked as -1 to null
runtimeAnalysis_df['Cell'].loc[runtimeAnalysis_df['Cell'] < 0 ] = np.nan

# set zero error cells to null
runtimeAnalysis_df['Error'].loc[runtimeAnalysis_df['Error'] == 0 ] = np.nan

In [ ]:
runtimeAnalysis_df

In [ ]:
outputLocation=Path(r'../reports/index.html')

In [ ]:
x = runtimeAnalysis_df['Time Stamp']
y = runtimeAnalysis_df['Error']
fig = px.scatter(runtimeAnalysis_df, x=x, y=y, color='Iterations')
fig.write_html(outputLocation)

In [ ]:
# sets path within hdf file to find cell locations
hdfGeo_geoCellCenter = hdfFile['Geometry']['2D Flow Areas'][geometry_2DName]['Cells Center Coordinate']

# creates dataframe from cell locations based on hdf file
geoCellCenter=pd.DataFrame(np.array(hdfGeo_geoCellCenter))

# renames columns to be easily identified
geoCellCenter=geoCellCenter.rename(columns={0:'X',1:'Y'})

# creates column with appropriate cell numbers
geoCellCenter['Cell']=range(0,len(geoCellCenter))

# creates geodataframe from dataframe using coordinates
geoCellCenter=gpd.GeoDataFrame(geoCellCenter,geometry=gpd.points_from_xy(geoCellCenter.X,geoCellCenter.Y,crs='EPSG:6479'))

In [ ]:
# sets path within hdf file to find cell itteration data
hdfResults_cellIterLook = hdfFile['Results']['Unsteady']['Output']['Output Blocks']['Base Output']['Summary Output']['2D Flow Areas'][geometry_2DName]['Cell Cumulative Iter Lookup']

# creates dataframe from cell itterations
cellIterLook = pd.DataFrame(np.array(hdfResults_cellIterLook),columns=['Cell','Iterations'])

In [ ]:
cellIterLook
select_cellLookup = cellIterLook['Cell'].tolist()

In [ ]:
geoCellIterLook = geoCellCenter[pd.DataFrame(geoCellCenter.Cell.tolist()).isin(select_cellLookup).any(1).values]

In [ ]:
geoCellIterLook.explore()

 ## hdf File for Sensitivity Analysis

In [ ]:
%run C:\Users\christianl\repos\WF_Analysis\ras_runtimeDF.py

In [ ]:
# sets working directory
pth=Path(r'C:\Users\christianl\Documents\WF_SensitivityAnalysis\WF_WestForkCalcasieu')
os.chdir(pth)

In [ ]:
# results files to be compared
ctrl00_laura2020 = 'WF_WestForkCalcasieu.p06.hdf'
gbrl40_laura2020 = 'WF_WestForkCalcasieu.p15.hdf'
gbrl50_laura2020 = 'WF_WestForkCalcasieu.p07.hdf'
gbrl50_10s_laura2020 = 'WF_WestForkCalcasieu.p10.hdf'
gbrl50_30s_laura2020 = 'WF_WestForkCalcasieu.p11.hdf'
gbrl50_60s_laura2020 = 'WF_WestForkCalcasieu.p12.hdf'
gbrl50_120s_laura2020 = 'WF_WestForkCalcasieu.p13.hdf'
filelist = [ctrl00_laura2020, gbrl40_laura2020, gbrl50_laura2020, gbrl50_10s_laura2020, gbrl50_30s_laura2020, gbrl50_60s_laura2020, gbrl50_120s_laura2020]

In [ ]:
df_list = [generate_runtimeDF(file) for file in filelist]

In [ ]:
dfs = {"ctrl00_laura2020": df_list[0],
"gbrl40_laura2020": df_list[1],
"gbrl50_laura2020": df_list[2],
"gbrl50_10s_laura2020": df_list[3],
"gbrl50_30s_laura2020": df_list[4],
"gbrl50_60s_laura2020": df_list[5],
"gbrl50_120s_laura2020": df_list[6]}

In [ ]:
# crete a dataframe with all the data with the name of the file as the column name in each row
df = pd.concat(dfs,axis=0, keys=dfs.keys()).reset_index()
df.head()

In [ ]:
df.keys()

In [ ]:
%%time
timeseries_df = datashade(hv.Scatter(df,kdims='Time Stamp',vdims=('Error', 'Error (%)'))).options(width=1000, height=500, title='Error vs Time Stamp', show_grid=True)
timeseries_df

In [ ]:
hmap = hv.HoloMap({file: hv.Scatter(dfs[file], kdims='Time Stamp', vdims=('Error', 'Error (%)')) for file in dfs.keys()}, kdims='File')

In [ ]:
hmap.opts(width=1000, height=500, title='Error vs Time Stamp', show_grid=True)

In [ ]:
# import plotly.graph_objects as go
# from plotly.offline import iplot

In [ ]:
# fig = px.scatter(x = df_list[1]["Time Stamp"], y= df_list[1]["Error"])
# fig.show()

In [ ]:
# fig = go.Figure()
# for idx, obj in enumerate(df_list):
#     fig = fig.add_trace(go.Scattergl(x = df_list[idx]["Time Stamp"],
#                                    y = df_list[idx]["Error"],
#                                    mode='markers'
#                                    ))
# fig.show()

In [ ]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output

In [ ]:
# dfs_keys = list(dfs.keys())
# dfs_keys[1]

In [ ]:
# dfs.get('gbrl40_laura2020')

In [ ]:
# # Build App with 
# app = JupyterDash(__name__)
# app.layout = html.Div([
#     html.H1("JupyterDash Demo"),
#     dcc.Graph(id='graph'),
#     html.Label([
#         "Input Variable",
#         dcc.Dropdown(
#             # Build options from list of dataframes
#             options=[{'label': k, 'value': k} for k in dfs.keys()],
#             value='gbrl40_laura2020', id='df-dropdown')
#     ]),
#     html.Label([
#         "X Variable",
#         dcc.Dropdown(
#             id='x-dropdown', clearable=False,
#             value='Time Step', options=[
#                 {'label': c, 'value': c}
#                 for c in df_list[0].columns
#             ])
#     ]),
#     html.Label([
#         "Y Variable",
#         dcc.Dropdown(
#             id='y-dropdown', clearable=False,
#             value='Error', options=[
#                 {'label': c, 'value': c}
#                 for c in df_list[0].columns
#             ])]
#         )

# ])
# # Define callback to update graph
# @app.callback(
#     Output('graph', 'figure'),
#     [Input("df-dropdown", "value"),
#         Input("x-dropdown", "value"),
#         Input("y-dropdown", "value")]
# )
# def update_figure(df, x, y):
#     return px.scatter(
#         dfs[df], x=x, y=y,
#         render_mode="webgl", title="Scatter Plot"
#     )
# # Run app and display result eternal to the notebook
# app.run_server(mode='external')

In [ ]:
# from IPython.display import IFrame
# documentation = IFrame(src='http://127.0.0.1:8050/', width=1000, height=500)
# display(documentation)

In [ ]:
# # Create a Dash app within the JupyterDash server that shows the cell locations and the number of iterations
# # for each cell in the model
# app = JupyterDash(__name__)
# app.layout = html.Div([
#     html.H1("JupyterDash Demo"),
#     dcc.Graph(id='graph'),
#     html.Label([
#         "Input Variable",
#         dcc.Dropdown(
#             # Build options from list of dataframes with the cell number and the number of iterations
#             options=[{'label': k, 'value': k} for k in dfs.keys()],
#             value='gbrl40_laura2020', id='df-dropdown')
#     ])
# ])
# # Define callback to update graph
# @app.callback(
#     Output('graph', 'figure'),
#     [Input("df-dropdown", "value")]
# )
# # create gespatial plot of cell locations and number of iterations
# def update_figure(df):
#     return px.scatter_geo(
#         dfs[df], lat="Latitude", lon="Longitude", color="Iterations",
#         render_mode="webgl", title="Scatter Plot"
#     )
# # Run app and display result eternal to the notebook
# app.run_server(mode='external')